In [1]:
%matplotlib inline
#
from lrgsglib import *
#
move_to_rootf(pathname='lrg_eegfc')
#
from lrg_eegfc import *

Current working directory: /home/opisthofulax/Documents/research+/brain_network/lrg_eegfc


## Data reader

In [2]:
data_dict, int_label_pos_map = load_data_dict(
    mat_path, patients_list, phase_labels, param_keys_list
)

In [ ]:
#
correlation_protocol = dict(threshold=0, spectral_cleaning=True)
#
path_figs = Path('data') / 'figures' / 'per_patient_preVSpost'
path_figs.mkdir(parents=True, exist_ok=True)
# Plot settings
kw_thline = dict(color='b', linestyle='--', label=r'$\mathcal{D}_{\rm th}$')
kw_draw_graph = dict(node_size=100, font_size=8, with_labels=True)
kw_dentro = dict(above_threshold_color='k', leaf_font_size=5, orientation='top')

In [7]:
ultraDist = {}
lnkgMat = {}
condDist = {}
#
for patient in patients_list:
    print(patient)
    #
    data_pat = data_dict[patient]
    pin_labels = int_label_pos_map[patient]['label']
    #
    ultraDist[patient] = {}
    lnkgMat[patient] = {}
    condDist[patient] = {}
    #
    for phase in phase_labels:
        print(phase, end='\t')
        #
        ultraDist[patient][phase] = {}
        lnkgMat[patient][phase] = {}
        condDist[patient][phase] = {}
        # Extract time series data for the current phase
        data_pat_phase = data_pat[phase]
        data_pat_phase_ts = data_pat_phase['data']
        fs = data_pat_phase['fs']
        for band, (low_freq, high_freq) in BRAIN_BANDS.items():
            print(band, flush=True, end='\r')
            #
            result = process_network_for_phase(
                data_pat_phase_ts, fs, band, correlation_protocol, 
                pin_labels, filter_order=1
            )
            G, label_dict, lnkgM, clTh, corr_mat, dists = result
            ultraDist[patient][phase][band] = extract_ultrametric_matrix(
                lnkgM,
                G.number_of_nodes()
            )
            lnkgMat[patient][phase][band] = lnkgM
            condDist[patient][phase][band] = dists

Pat_03


/home/opisthofulax/Documents/research+/brain_network/lrg_eegfc/src/lrg_eegfc/utils/corrmat.py:600: RuntimeWarning: Reducing filter order from 4 to 2 for narrow band (0.53-4 Hz, 0.0068 of Nyquist) to avoid numerical instabilities.
  warnings.warn(
/home/opisthofulax/Documents/research+/brain_network/lrg_eegfc/lrgsglib/src/lrgsglib/utils/basic/signals.py:58: RuntimeWarning: Very narrow filter bandwidth (0.0068 of Nyquist). Consider reducing filter order or increasing bandwidth.
  warnings.warn(


/home/opisthofulax/Documents/research+/brain_network/lrg_eegfc/src/lrg_eegfc/utils/corrmat.py:600: RuntimeWarning: Reducing filter order from 4 to 2 for narrow band (4-8 Hz, 0.0078 of Nyquist) to avoid numerical instabilities.
  warnings.warn(
/home/opisthofulax/Documents/research+/brain_network/lrg_eegfc/lrgsglib/src/lrgsglib/utils/basic/signals.py:58: RuntimeWarning: Very narrow filter bandwidth (0.0078 of Nyquist). Consider reducing filter order or increasing bandwidth.
  warnings.warn(


/home/opisthofulax/Documents/research+/brain_network/lrg_eegfc/src/lrg_eegfc/utils/corrmat.py:600: RuntimeWarning: Reducing filter order from 4 to 2 for narrow band (8-13 Hz, 0.0098 of Nyquist) to avoid numerical instabilities.
  warnings.warn(
/home/opisthofulax/Documents/research+/brain_network/lrg_eegfc/lrgsglib/src/lrgsglib/utils/basic/signals.py:58: RuntimeWarning: Very narrow filter bandwidth (0.0098 of Nyquist). Consider reducing filter order or increasing bandwidth.
  warnings.warn(


/home/opisthofulax/anaconda3/envs/lapbrain/lib/python3.12/site-packages/scipy/signal/_filter_design.py:4334: RuntimeWarning: overflow encountered in scalar negative
  m = numpy.arange(-N+1, N, 2)


Pat_02ammatataa
Pat_02delta


/home/opisthofulax/Documents/research+/brain_network/lrg_eegfc/src/lrg_eegfc/utils/corrmat.py:600: RuntimeWarning: Reducing filter order from 4 to 2 for narrow band (0.53-4 Hz, 0.0034 of Nyquist) to avoid numerical instabilities.
  warnings.warn(
/home/opisthofulax/Documents/research+/brain_network/lrg_eegfc/lrgsglib/src/lrgsglib/utils/basic/signals.py:58: RuntimeWarning: Very narrow filter bandwidth (0.0034 of Nyquist). Consider reducing filter order or increasing bandwidth.
  warnings.warn(


/home/opisthofulax/Documents/research+/brain_network/lrg_eegfc/src/lrg_eegfc/utils/corrmat.py:600: RuntimeWarning: Reducing filter order from 4 to 2 for narrow band (4-8 Hz, 0.0039 of Nyquist) to avoid numerical instabilities.
  warnings.warn(
/home/opisthofulax/Documents/research+/brain_network/lrg_eegfc/lrgsglib/src/lrgsglib/utils/basic/signals.py:58: RuntimeWarning: Very narrow filter bandwidth (0.0039 of Nyquist). Consider reducing filter order or increasing bandwidth.
  warnings.warn(


/home/opisthofulax/Documents/research+/brain_network/lrg_eegfc/src/lrg_eegfc/utils/corrmat.py:600: RuntimeWarning: Reducing filter order from 4 to 2 for narrow band (8-13 Hz, 0.0049 of Nyquist) to avoid numerical instabilities.
  warnings.warn(
/home/opisthofulax/Documents/research+/brain_network/lrg_eegfc/lrgsglib/src/lrgsglib/utils/basic/signals.py:58: RuntimeWarning: Very narrow filter bandwidth (0.0049 of Nyquist). Consider reducing filter order or increasing bandwidth.
  warnings.warn(


In [ ]:
from itertools import combinations
import pandas as pd
from lrgsglib.utils.basic.linalg import compare_ultrametric_trees

# Tweak these if you want to restrict the analysis
bands_to_check = list(BRAIN_BANDS.keys())
phases_to_compare = list(phase_labels)

pairwise_stats: list[dict] = []

for patient in patients_list:
    for band in bands_to_check:
        for phase_a, phase_b in combinations(phases_to_compare, 2):
            # Skip missing results gracefully
            try:
                U1 = ultraDist[patient][phase_a][band]
                U2 = ultraDist[patient][phase_b][band]
            except KeyError:
                continue

            metrics = compare_ultrametric_trees(
                U1,
                U2,
                scale="log",          # options: 'linear', 'log', 'zscore', 'rank'
                value_metric="euclidean",
                normalize=True,
            )

            pairwise_stats.append({
                "patient": patient,
                "band": band,
                "phase_a": phase_a,
                "phase_b": phase_b,
                **metrics,
            })

comparisons_df = pd.DataFrame(pairwise_stats)
display(comparisons_df.sort_values(["patient", "band", "phase_a", "phase_b"]))


In [ ]:
band = 'delta'
n = len(phase_labels)
distance_matrix = np.zeros((n, n))
for i, phase in enumerate(phase_labels):
    for j, phase2 in enumerate(phase_labels):
        if i < j:
            d = ultrametric_distance_permutation_robust(
                lnkgMat[phase][band], 
                lnkgMat[phase2][band], 
                condDist[phase][band], 
                condDist[phase2][band], 
                pin_labels, 
                metric='euclidean')
            distance_matrix[i, j] = d
            distance_matrix[j, i] = d

In [ ]:


# Create figure with GridSpec for custom layout
fig = plt.figure(figsize=(16, 8))
gs = gridspec.GridSpec(2, 6, figure=fig, hspace=0.3, wspace=0.3)

# Left subplot for distance matrix - now spans 2 columns instead of 1
ax_dist = fig.add_subplot(gs[:, 0:2])
im = ax_dist.imshow(distance_matrix, cmap='viridis')
ax_dist.set_title('Distance Matrix')
ax_dist.set_xticks(range(len(phase_labels)))
ax_dist.set_yticks(range(len(phase_labels)))
ax_dist.set_xticklabels(phase_labels, rotation=45)
ax_dist.set_yticklabels(phase_labels)
div, cax, clb = imshow_colorbar_caxdivider(im, ax_dist)
# plt.colorbar(im, ax=ax_dist)

# Right subplots for dendrograms (2x2 grid) - now starting from column 2
for i, phase in enumerate(phase_labels):
    row = i // 2
    col = i % 2 + 2  # Start from column 2 (after distance matrix)
    if col == 2:
        ax_dend = fig.add_subplot(gs[row, 2:4])  # Span 2 columns for first dendrogram in each row
    else:
        ax_dend = fig.add_subplot(gs[row, 4:6])  # Span 2 columns for second dendrogram in each row
    
    lnkg = lnkgMat[phase][band]
    Z_opt = optimal_leaf_ordering(lnkg, condDist[phase][band])
    dendrogram(Z_opt, ax=ax_dend, no_labels=True, **kw_dentro)
    ax_dend.set_yscale('log')
    tmin = lnkg[:, 2][0] * 0.8
    tmax = lnkg[:, 2][-1] * 1.01
    ax_dend.set_ylim(tmin, tmax)
    ax_dend.set_title(f'{phase}')

plt.show()

### One Patient, one phase

In [ ]:
patient = 'Pat_02'
phase = 'taskLearn'
#
corr_net_filt = dict(threshold=0)
#
data_pat_phase = data_dict[patient][phase]
data_pat_phase_ts = data_pat_phase['data']
fs = data_pat_phase['fs']
filter_order = data_pat_phase['filter_order']
#
n_intervals = 50
interval_length = int(data_pat_phase_ts.shape[1] // n_intervals)
n_intervals, interval_length

In [ ]:
corr_mat_band_tw = {}
for band_name, (low, high) in tqdm(BRAIN_BANDS.items()):
    corr_mat_band_tw[band_name] = {}
    for i in range(n_intervals):
        start = i * interval_length
        end = (i + 1) * interval_length
        interval_data = data_pat_phase_ts[:, start:end]
        # pay attention the filter is not chnaging if it fails (filter order?)
        filter_data = bandpass_sos(interval_data, low, high, fs, 1)
        corr_mat = build_corr_network(filter_data, threshold=0)
        
        # Find EXACT detachment threshold
        Th, jumps = find_threshold_jumps(nx.from_numpy_array(corr_mat))
        
        corr_mat = build_corr_network(filter_data, threshold=Th[jumps[0]])
        corr_mat_band_tw[band_name][i] = corr_mat

Given the LRG induced ultrametric distances 
$$
    \mathcal{D}_{ij}(\tau) = \frac{1 - \delta_{ij}}{K_{ij}(\tau)},
$$
between the nodes in the network built in a time window, one can measure the (Euclidean) distance of distances
$$
    d(\mathcal{D}_{ij}^{t_1}(\tau), \mathcal{D}_{ij}^{t_2}(\tau))
$$
Maybe we want to consider the integral of this quantity ($\int d(\tau)\mathop{\rm d\tau}$) and maybe weight it on the relevant scales of the system.

In [ ]:
# Example usage with your linkage matrix:
cc = corr_mat_band_tw['low_gamma'][49]
gg = nx.from_numpy_array(cc)
n_nodes = gg.number_of_nodes()
linkage_matrix = MakeLinkageMatrix(gg, tau=10)
ud = extract_ultrametric_matrix(linkage_matrix, n_nodes)
sm1,dS,vL,ttts = entropy(gg, t2=3)
# Visualize the ultrametric distance matrix
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].plot(ttts[1:], dS, '-', label='Entropy')
ax[0].set_xscale('log')
inset_ax = ax[0].inset_axes([0.4, 0.4, 0.7, 0.7])  # [x, y, width, height]
ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)
pos = nx.nx_agraph.graphviz_layout(gg)
nx.draw(gg, ax=inset_ax, with_labels=True, node_size=50, font_size=8)
im = ax[1].imshow(ud, cmap='viridis')
fig.colorbar(im, ax=ax[1], label='Ultrametric Distance')
plt.show()

In [ ]:
ultraDist = []
for i in range(n_intervals):
    cc = corr_mat_band_tw['delta'][i]
    gg = nx.from_numpy_array(cc)
    n_nodes = gg.number_of_nodes()
    linkage_matrix = MakeLinkageMatrix(gg, tau=3)
    ultraDist.append(extract_ultrametric_matrix(linkage_matrix, n_nodes))

In [ ]:
# Example: compute pairwise distances between all ultrametric matrices
n = len(ultraDist)
distance_matrix = np.zeros((n, n))
for i in range(n):
    for j in range(i+1, n):
        d = ultrametric_matrix_distance(ultraDist[i], ultraDist[j])
        distance_matrix[i, j] = d
        distance_matrix[j, i] = d
distance_matrix /= np.max(distance_matrix)
# plt.imshow(distance_matrix)
# plt.colorbar()
for i in range(10):
    plt.plot(distance_matrix[i], 'o:', label=f'interval {i}')

In [ ]:
plt.imshow(distance_matrix, interpolation='none', cmap='binary')
plt.colorbar()

In [ ]:
i1 = 0
i2 = 12

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(ultraDist[i1])
ax[1].imshow(ultraDist[i2])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(corr_mat_band_tw['delta'][i1])
ax[1].imshow(corr_mat_band_tw['delta'][i2])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

cc0 = corr_mat_band_tw['delta'][i1]
cc1 = corr_mat_band_tw['delta'][i2]
gg0 = nx.from_numpy_array(cc0)
gg1 = nx.from_numpy_array(cc1)

pos0 = nx.nx_agraph.graphviz_layout(gg0)
pos1 = nx.nx_agraph.graphviz_layout(gg1)

nx.draw(gg0, pos=pos0, ax=ax[0], with_labels=True, node_size=50, font_size=8)
nx.draw(gg1, pos=pos0, ax=ax[1], with_labels=True, node_size=50, font_size=8)